In [1]:
import numpy as np
import tensorflow as tf
import sys
from sklearn.model_selection import train_test_split

In [12]:
x = np.load("../input/Sign-language-digits-dataset/X.npy")
y = np.load("../input/Sign-language-digits-dataset/Y.npy")

x = x.reshape(x.shape[0], 64, 64, 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [3]:
print x.shape, x.dtype
print y.shape

(2062, 64, 64, 1) float32
(2062, 10)


In [4]:
# Hyperparameters
num_filters = [64,128,128,256]
strides = [1,2]
image_height, image_width, image_channels = 64,64,1
learning_rate = 1e-3
keep_prob = 0.7
num_hidden = 128
num_epochs = 300
log_every = 10
test_every = 10
save_every = 50
batch_size = 64

In [5]:
def rprint(string):
    sys.stdout.write("\r{}".format(string))
    sys.stdout.flush()

In [6]:
def conv_layer(name, x, filter_size, input_channels, output_channels, strides=(1, 1), padding='SAME'):
    with tf.variable_scope(name):
        filters = tf.Variable(tf.random_normal([filter_size[0], filter_size[1], input_channels, output_channels]), dtype=tf.float32, name="filters")
        bias = tf.Variable(tf.random_normal([output_channels]),dtype=tf.float32,name="bias")
        conv2d_op = tf.nn.conv2d(input=x, filter=filters, strides=[1, strides[0], strides[1], 1], padding=padding)
        conv2d_op = tf.nn.bias_add(conv2d_op, bias)
        return conv2d_op

In [7]:
def max_pool_layer(x, kernel_shape, strides=(2,2), padding="SAME"):
    max_pool_op = tf.nn.max_pool(value=x,
                                 ksize=[1, kernel_shape[0], kernel_shape[1], 1], 
                                 strides=[1,strides[0], strides[1], 1], 
                                 padding=padding, 
                                 name="maxpool"
                                )
    return max_pool_op

In [8]:
def leaky_relu(x, leakiness=0.0):
    return tf.nn.leaky_relu(features=x, alpha=leakiness, name="leaky_relu")

In [9]:
def build_model(x,dropout_keepprob):
    with tf.variable_scope("CNN"):
        with tf.variable_scope("layer-1"):
            x = conv_layer("cnn-1",x,(3,3),image_channels,num_filters[0])
            x = leaky_relu(x,0.01)
            x = max_pool_layer(x,(2,2))
        print "layer-1",x.get_shape().as_list()
        
        with tf.variable_scope("layer-2"):
            x = conv_layer("cnn-2",x,(3,3),num_filters[0],num_filters[1])
            x = leaky_relu(x,0.01)
            x = max_pool_layer(x,(2,2))
        print "layer-2",x.get_shape().as_list()
        
        with tf.variable_scope("layer-3"):
            x = conv_layer("cnn-3",x,(3,3),num_filters[1],num_filters[2])
            x = leaky_relu(x,0.01)
            x = max_pool_layer(x,(2,2))
        print "layer-3",x.get_shape().as_list()
        
        with tf.variable_scope("layer-4"):
            x = conv_layer("cnn-4",x,(3,3),num_filters[2],num_filters[3])
            x = leaky_relu(x,0.01)
            x = max_pool_layer(x,(2,2))
        print "layer-4",x.get_shape().as_list()
    
    x_shape = x.get_shape().as_list()[1:]
    with tf.variable_scope("FC"):
        with tf.variable_scope("layer-1"):
            weights = tf.Variable(tf.random_normal([np.prod(x_shape),2048]), dtype=tf.float32, name="weights-1")
            biases = tf.Variable(tf.random_normal([2048]), dtype=tf.float32, name="biases-1")
            
            x = tf.reshape(x,[-1,np.prod(x_shape)])
            x = leaky_relu(tf.matmul(x,weights)+biases)
            x = tf.nn.dropout(x,dropout_keepprob)
        
        with tf.variable_scope("layer-2"):
            weights = tf.Variable(tf.random_normal([2048,1024]), dtype=tf.float32, name="weights-2")
            biases = tf.Variable(tf.random_normal([1024]), dtype=tf.float32, name="biases-2")
            
#             x = tf.reshape(x,[-1,np.prod(x_shape)])
            x = leaky_relu(tf.matmul(x,weights)+biases)
            x = tf.nn.dropout(x,dropout_keepprob)
        
        with tf.variable_scope("layer-3"):
            weights = tf.Variable(tf.random_normal([1024,10]), dtype=tf.float32, name="weights-3")
            biases = tf.Variable(tf.random_normal([10]), dtype=tf.float32,name="biases-3")
            
            x = tf.matmul(x,weights)+biases
            
        
    return x

In [10]:
def build_output_ops(logits,labels):
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss_op)
    correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return [train_op, loss_op, accuracy]

In [11]:
X = tf.placeholder(tf.float32, shape=[None, image_height, image_width, image_channels], name="inputs")
Y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")
dropout_keepprob = tf.placeholder_with_default(1.0,shape=(), name="dropout_keepprob")

logits = build_model(X,dropout_keepprob)
train_ops = build_output_ops(logits,Y)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1,num_epochs+1):
        for batch_index in range(0,len(x_train),batch_size):
            feed_dict = {X:x_train[batch_index:batch_index+batch_size], Y:y_train[batch_index:batch_index+batch_size], dropout_keepprob:keep_prob}
            _, loss_val, accuracy_val = sess.run(train_ops, feed_dict=feed_dict)
            rprint("Epoch: {}, Batch: {:0>4d}/{}, Loss: {:.2f}, Accuracy: {:.2%}".format(epoch,batch_index,len(x),loss_val,accuracy_val))
        if epoch%test_every==0:
            accuracies = []
            for batch_index in range(0,len(x_test),batch_size):
                feed_dict = {X:x_test[batch_index:batch_index+batch_size], Y:y_test[batch_index:batch_index+batch_size]}
                accuracy_val = sess.run(train_ops[2], feed_dict=feed_dict)
                accuracies.append(accuracy_val)
            print
            print "Test Accuracy after {} epochs: {:.2%}".format(epoch,np.mean(accuracies))
        

layer-1 [None, 32, 32, 64]
layer-2 [None, 16, 16, 128]
layer-3 [None, 8, 8, 128]
layer-4 [None, 4, 4, 256]
Epoch: 10, Batch: 1600/2062, Loss: 31350434.00, Accuracy: 46.94%%
Test Accuracy after 10 epochs: 68.60%
Epoch: 20, Batch: 1600/2062, Loss: 5274439.00, Accuracy: 75.51%%
Test Accuracy after 20 epochs: 82.63%
Epoch: 30, Batch: 1600/2062, Loss: 2697803.50, Accuracy: 85.71%%
Test Accuracy after 30 epochs: 84.91%
Epoch: 40, Batch: 1600/2062, Loss: 3275653.25, Accuracy: 81.63%
Test Accuracy after 40 epochs: 85.22%
Epoch: 50, Batch: 1600/2062, Loss: 1842920.50, Accuracy: 89.80%
Test Accuracy after 50 epochs: 87.68%
Epoch: 60, Batch: 1600/2062, Loss: 1752399.62, Accuracy: 89.80%
Test Accuracy after 60 epochs: 89.37%
Epoch: 70, Batch: 1600/2062, Loss: 1189098.50, Accuracy: 91.84%
Test Accuracy after 70 epochs: 87.54%
Epoch: 80, Batch: 1600/2062, Loss: 2029267.25, Accuracy: 89.80%
Test Accuracy after 80 epochs: 90.98%
Epoch: 90, Batch: 1600/2062, Loss: 1013782.75, Accuracy: 89.80%
Test Accu